1. What fraction of assessments are for properties of the most common class? For now, consider all the assessments, even though some properties may be listed more than once.

    A: Common Class / Total records

2. What is the median assessed improvement value, considering only non-zero assessments? Consider only the latest assessment value for each property, which is uniquely identified by the "Block and Lot Number" column.

    A: 
    median(Latest prop)

3. Calculate the average improvement value (using only non-zero assessments) in each neighborhood. What is the difference between the greatest and least average values?

    sort latest prop.
    for each neighbourhood, calc, median: max - min

4. What is the yearly growth rate of Land Values over the years covered by this data? Take a simplistic model: the value is given by P=P0ert, where is measured in years.
    
    (A more complete model would give each property its own base price .) Estimate over all assessments with a non-zero land value. (Hint: Consider using linear regression and logarithms.)
    
5. We can use the property locations to estimate the areas of the neighborhoods. Represent each as an ellipse with semi axes given by a single standard deviation of the longitude and latitude. What is the area, in square kilometers, of the largest neighborhood measured in this manner? Be sure to filter out invalid coordinates.

6. Considering only properties with non-zero numbers of bedrooms and units, calculate the average number of bedrooms per unit in each zip code. Use the ratio of the means instead of the mean of the ratio. What is this ratio in the zip code where it achieves its maximum?


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

d = pd.read_csv("Historic_Secured_Property_Tax_Rolls.csv", low_memory=False)
print(d.shape)

d.iloc[0]

# Making sure no record without ID
d['Block and Lot Number'].isnull().sum()

(1612110, 43)


0

In [2]:
'''
1. What fraction of assessments are for properties of the most common class? 
For now, consider all the assessments, even though some properties may be listed more than once.
'''

'\n1. What fraction of assessments are for properties of the most common class? \nFor now, consider all the assessments, even though some properties may be listed more than once.\n'

In [3]:
common_class = d[['Block and Lot Number', 'Property Class Code']].groupby('Property Class Code').agg('count')
print('A. Common Class (Excluding NAs): {0:.10f}'.format(\
    (max(common_class['Block and Lot Number'])/sum(common_class['Block and Lot Number']))))
print('A. Common Class (Including NAs): {0:.10f}'.format(\
    (max(common_class['Block and Lot Number'])/d['Block and Lot Number'].count())))

A. Common Class (Excluding NAs): 0.4709759864
A. Common Class (Including NAs): 0.4707253227


In [4]:
'''
2. What is the median assessed improvement value, considering only non-zero assessments? 
Consider only the latest assessment value for each property, which is uniquely identified 
by the "Block and Lot Number" column.
'''

'\n2. What is the median assessed improvement value, considering only non-zero assessments? \nConsider only the latest assessment value for each property, which is uniquely identified \nby the "Block and Lot Number" column.\n'

In [5]:
import statistics as stat
d_improvement = d[d['Closed Roll Assessed Improvement Value'] != 0]
sliced = d_improvement.groupby(['Block and Lot Number'])['Closed Roll Fiscal Year'].transform(max) == d_improvement['Closed Roll Fiscal Year']

med = stat.median(d_improvement.ix[sliced,'Closed Roll Assessed Improvement Value'])
print('B. Median improvement value: {0:.10f}'.format(med))

B. Median improvement value: 209161.0000000000


In [6]:
'''
3. Calculate the average improvement value (using only non-zero assessments) in 
each neighborhood. What is the difference between the greatest and least average values?
'''

'\n3. Calculate the average improvement value (using only non-zero assessments) in \neach neighborhood. What is the difference between the greatest and least average values?\n'

In [7]:
means = d_improvement.loc[sliced].groupby(['Neighborhood Code'])['Closed Roll Assessed Improvement Value'].mean()

print('C. Average improvement: {0:.10f}'.format(max(means) - min(means)))

C. Average improvement: 5071144.8168576164


In [8]:
'''
What is the yearly growth rate of Land Values over the years covered by this data? 
Take a simplistic model: the value is given by P=P0e^rt, where t is measured in years.
    
(A more complete model would give each property its own base price .) 
Estimate over all assessments with a non-zero land value. (Hint: Consider using 
linear regression and logarithms.)
'''

'\nWhat is the yearly growth rate of Land Values over the years covered by this data? \nTake a simplistic model: the value is given by P=P0e^rt, where t is measured in years.\n    \n(A more complete model would give each property its own base price .) \nEstimate over all assessments with a non-zero land value. (Hint: Consider using \nlinear regression and logarithms.)\n'

In [9]:
d_land = d[d['Closed Roll Assessed Land Value'] != 0]
d_land['x'] = (d_land['Closed Roll Fiscal Year'] - min(d_land['Closed Roll Fiscal Year']-1))
d_land['y'] = np.log(d_land['Closed Roll Assessed Land Value'])
x = d_land[d_land['y']>0]['x'].values
y = d_land[d_land['y']>0]['y'].values
[m, b]= np.polynomial.polynomial.polyfit(x, y,1)
print("Annual growth rate: {0:.10f}".format(m/100))

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Annual growth rate: 0.1163263467


In [10]:
'''
5. We can use the property locations to estimate the areas of the neighborhoods. 
Represent each as an ellipse with semi axes given by a single standard deviation of the longitude and latitude. 
What is the area, in square kilometers, of the largest neighborhood measured in this manner? 
Be sure to filter out invalid coordinates.

(long lat)
'''


'\n5. We can use the property locations to estimate the areas of the neighborhoods. \nRepresent each as an ellipse with semi axes given by a single standard deviation of the longitude and latitude. \nWhat is the area, in square kilometers, of the largest neighborhood measured in this manner? \nBe sure to filter out invalid coordinates.\n\n(long lat)\n'

In [11]:
import math 
d_location = d[pd.notnull(d['Location'])]

d_location.loc[:,'Lat'] = [float(i.split(',')[0]) for i in d_location['Location'].map(lambda x: x.strip('()'))]
d_location.loc[:,'Long'] = [float(i.split(',')[1]) for i in d_location['Location'].map(lambda x: x.strip('()'))]

d_locSTD = d_location.groupby(['Neighborhood Code'])['Lat', 'Long'].agg(np.std)
d_locSTD.loc[:,'y_axes'] = d_locSTD.apply(lambda x: math.cos(x['Long'])*111.320*0.5, axis=1)
d_locSTD.loc[:,'area'] = math.pi*(d_locSTD['Lat']*110.574*0.5)*d_locSTD['y_axes']

print('Largest area: {0:.10f} squared km.'.format(float(d_locSTD.loc[d_locSTD['area'].idxmax(),'area'])))

/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Largest area: 107.7053961510 squared km.


In [12]:
'''
6. Considering only properties with non-zero numbers of bedrooms and units, 
calculate the average number of bedrooms per unit in each zip code. 
Use the ratio of the means instead of the mean of the ratio. 
What is this ratio in the zip code where it achieves its maximum?
'''

'\n6. Considering only properties with non-zero numbers of bedrooms and units, \ncalculate the average number of bedrooms per unit in each zip code. \nUse the ratio of the means instead of the mean of the ratio. \nWhat is this ratio in the zip code where it achieves its maximum?\n'

In [13]:
d_room = d[d.groupby(['Block and Lot Number'])['Closed Roll Fiscal Year'].transform(max) == d['Closed Roll Fiscal Year']]
d_room = d_room[:][(d_room['Number of Bedrooms'] != 0) & d_room['Number of Units'] != 0]


avg = d_room.groupby(['Zipcode of Parcel'])['Number of Bedrooms', 'Number of Units'].agg(np.mean)
avg = avg['Number of Bedrooms']/avg['Number of Units']

print('Average bedroom per unit: {0:.10f}'.format(max(avg)))

Average bedroom per unit: 3.5990437158


In [14]:
'''
What is the difference between the average number of units in buildings build in or after 1950, 
and that for buildings built before 1950? Consider only buildings that have non-zero units reported, 
and ignore buildings with obviously incorrect years. 
To try to avoid the effect of improvements to buildings, use the earliest record for 
each property, not the latest.
'''

'\nWhat is the difference between the average number of units in buildings build in or after 1950, \nand that for buildings built before 1950? Consider only buildings that have non-zero units reported, \nand ignore buildings with obviously incorrect years. \nTo try to avoid the effect of improvements to buildings, use the earliest record for \neach property, not the latest.\n'

In [15]:
d_unit = d.loc[(d['Number of Units'] != 0) &  d['Year Property Built'].notnull(),:]
d_unit = d_unit[d_unit.groupby(['Block and Lot Number'])['Closed Roll Fiscal Year'].transform(min) == d_unit['Closed Roll Fiscal Year']]
q75 = d_unit['Year Property Built'].quantile(0.75)
q25 = d_unit['Year Property Built'].quantile(0.25)
d_unit = d_unit.loc[(d_unit['Year Property Built'] > q25) & (d_unit['Year Property Built'] < 2016), :]
avg_b50 = np.mean(d_unit.loc[d_unit['Year Property Built'] < 1950, 'Number of Units'])
avg_a50 = np.mean(d_unit.loc[d_unit['Year Property Built'] > 1950, 'Number of Units'])
print('Difference avg unit before after 50s: {0:.10f}'.format(avg_a50 - avg_b50))

Difference avg unit before after 50s: 0.8985516241


In [16]:
'''
Estimate how built-up each zip code is by comparing the total property area to the total lot area. 
What is the largest ratio of property area to surface area of all zip codes?

'''


'\nEstimate how built-up each zip code is by comparing the total property area to the total lot area. \nWhat is the largest ratio of property area to surface area of all zip codes?\n\n'

In [17]:
d_area = d.loc[d['Lot Area'].notnull() & d['Lot Area'] != 0 & d['Property Area in Square Feet'].notnull()]
d_area = d_area[d_area.groupby(['Block and Lot Number'])['Closed Roll Fiscal Year'].transform(max) == d_area['Closed Roll Fiscal Year']]
d_area = d_area.groupby(['Zipcode of Parcel'])['Lot Area', 'Property Area in Square Feet'].\
agg(np.sum)
d_area['avg'] = d_area['Property Area in Square Feet']/d_area['Lot Area']
print('Ratio property to lot area: {0:.10f}'.format(max(d_area['avg'])))

Ratio property to lot area: 12.0603296061
